In [12]:
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 
from selenium.webdriver.common.by import By

## RAMAYANA SCRAPING

In [3]:
sku = #product > li:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1) 
name = #product > li:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)
image = #product > li:nth-child(1) > div:nth-child(1) > div:nth-child(1) > a:nth-child(1) > img:nth-child(1)
origin_price = #product > li:nth-child(1) > div:nth-child(1) > div:nth-child(3)
discounted_price = origin_price
category = #product > li:nth-child(103) > div:nth-child(1) > div:nth-child(4) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)

SyntaxError: invalid syntax (<ipython-input-3-482d460e34dc>, line 1)

In [13]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [14]:
# Scraping the page
def scrape_detail_page(url,collection):
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.ramayana
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]

    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    
    # Scrap page
    time.sleep(10)
    result = pd.DataFrame()
    image = 0
    finish = False
    wait = 0
    newHeight = 0
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while (wait < 10):
        newHeight = driver.execute_script("return document.body.scrollHeight")
        time.sleep(5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        lastHeight = driver.execute_script("return document.body.scrollHeight")
        if(newHeight>=lastHeight):
            wait = wait+1
            time.sleep(30)
            print(wait)
        else:
            wait = 0
            
    time.sleep(5)
    path = driver.find_elements_by_css_selector('#product > li')
    #print('path: '+ str(len(path)))
    for x in range(len(path)):
        try:
            name = path[x].find_element_by_css_selector('.product-image > a > img').get_attribute('title')
            image = path[x].find_element_by_css_selector('.product-image > a > img').get_attribute('src')
            sku = image
            price = re.sub("\.","",re.sub("Rp","",re.sub("\,-","",path[x].find_element_by_css_selector('.product-price').text)))

            cat = path[x].find_elements_by_css_selector('.product-tags > li > .product-tag')
            category = cat[0].text
            for x in range(1,len(cat)):
                text = cat[x].text
                if(len(text) > 0):
                    category = category + ";" + text

            # Return result
            date = datetime.datetime.now().strftime("%Y-%m-%d")
            new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':price, 'discounted_price':price, 'image':image,'category':category,'scraping_date':date}])
            result = result.append(new_result)
            
            # save to mongodb
            collection.insert_many(new_result.to_dict('records'))
            
        except:
            #PrintException()
            pass
    return result


In [15]:
# MAIN
#result = scrape_detail_page('http://www.ramayana.co.id/collection/category/anak','pakaian_anak')
#result = scrape_detail_page('http://www.ramayana.co.id/collection/category/wanita','pakaian_wanita')
#result = scrape_detail_page('http://www.ramayana.co.id/collection/category/pria','pakaian_pria')
result = scrape_detail_page('http://www.ramayana.co.id/collection/category_tag/sepatu','sepatu')

Connected successfully!!!
1
2
3
4
5
6
7
8
9
10


In [89]:
result

,category,discounted_price,image,name,origin_price,scraping_date,sku
0,PRIA;BAJU KOKO,179900,http://www.ramayana.co.id/userdata/product/res...,Kemeja Koko Two Tone RAF (07944521),179900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;WANITA,199900,http://www.ramayana.co.id/userdata/product/res...,Tas Canvas Backpack Diesel (07925833),199900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;WANITA,149900,http://www.ramayana.co.id/userdata/product/res...,Tas Ransel Aerosmith Bag (07902688),149900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;WANITA;TRAVEL BAG,199900,http://www.ramayana.co.id/userdata/product/res...,Tas Canvas Backpack Diesel (07925831),199900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;WANITA;TRAVEL BAG,50000,http://www.ramayana.co.id/userdata/product/res...,Tas Ransel School Pineapple (07913040),50000,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;WANITA;BACKPACK,199900,http://www.ramayana.co.id/userdata/product/res...,Tas Ransel Backpack Diesel (07925829),199900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;WANITA;TRAVEL BAG,199900,http://www.ramayana.co.id/userdata/product/res...,Tas Canvas Backpack Diesel (07925832),199900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;SWEATER,129900,http://www.ramayana.co.id/userdata/product/res...,Sweater Pria Absolute Raf21 (07917927),129900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA;SWEATSHIRT,109900,http://www.ramayana.co.id/userdata/product/res...,Kaus Pria Lengan Panjang Two Tone (07921973),109900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
0,PRIA,169900,http://www.ramayana.co.id/userdata/product/res...,Jaket Pria Hoodie Raf21 (07920253),169900,2019-07-14,http://www.ramayana.co.id/userdata/product/res...
